In [3]:
from electricitycostcalculator.electricity_rate_manager.rate_manager import ElectricityRateManager
from electricitycostcalculator.openei_tariff.openei_tariff_analyzer import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from calendar import monthrange

Use this notebook for
- testing the TOU-Tier functionality


In [4]:
def utc_to_local(data, local_zone="America/Los_Angeles"):
    '''
    This method takes in pandas DataFrame and adjusts index according to timezone in which is requested by user
    '''

    data = data.tz_convert(local_zone)  # accounts for localtime shift
    # Gets rid of extra offset information so can compare with csv data
    data = data.tz_localize(None)

    return data

# TOU functionality 

In [5]:
#TOU tests
__author__ = 'Olivier Van Cutsem'

from electricitycostcalculator.electricity_rate_manager.rate_manager import ElectricityRateManager
from electricitycostcalculator.openei_tariff.openei_tariff_analyzer import *
import pandas as pd

# ----------- TEST DEMO -------------- #
READ_FROM_JSON = False

# useful functions


if __name__ == '__main__':

    meter_uuid = 'e9c51ce5-4aa1-399c-8172-92073e273a0b'

    #
    ### Reading power consumption data
    #

    print("--- Loading meter data ...")
    df = pd.read_csv('meter.csv', index_col=0)  # import times series energy data for meters
    df.index.name = 'Time'
    df = df.set_index(pd.to_datetime(df.index, infer_datetime_format=True, utc=True))
    df["date"] = df.index

    data_meter = df[meter_uuid]
    data_meter = utc_to_local(data_meter, local_zone="America/Los_Angeles")

    #
    ### Reading OpenEI-based tariff rates, and binding it to the ElectricityRateManager
    #

    print("--- Calling OpenEI API ...")

    # (a) Example of using the OpenEI WEB API to populate tariff_data

    tariff_openei_apidata = OpenEI_tariff(utility_id='14328',
                                       sector='Commercial',
                                       tariff_rate_of_interest='E-19',
                                       distrib_level_of_interest=None,
                                       # it is at the secondary level, so not specified in the name
                                       phasewing=None,
                                       # the word 'Poly' is to be excluded, because the names may omit this info ..
                                       tou=True, tier_tou = False,
                                       option_exclusion=['(X)', '(W)', 'Poly'])  # Need to reject the option X and W

    tariff_openei_apidata.call_api(store_as_json=True)
        #tariff data
        
    # (b) Example of reading local tariff data, binding to elec_rate_handler
    #encoded according to OpenEI structure


    elecrate_manager = ElectricityRateManager() #elec_rate_handler

    # Binding an instance of ElectricityRateManager to a specific OpenEI tariff

    tariff_struct_from_openei_data(tariff_openei_apidata, elecrate_manager)  # This analyses the raw data from the openEI request and populate the "CostCalculator" object

    # BILLING PERIOD
    start_date_bill = datetime(2017, 8, 1, hour=0, minute=0, second=0)
    end_date_bill = datetime(2017, 8, 30, hour=23, minute=59, second=59)
    mask = (data_meter.index >= start_date_bill) & (data_meter.index <= end_date_bill)
    data_meter = data_meter.loc[mask]
    data_meter = data_meter.fillna(0)

    # 1) Get the bill over the period

    print("Calculating the bill for the period {0} to {1}".format(start_date_bill, end_date_bill))
    bill = elecrate_manager.compute_bill(data_meter, monthly_detailed=True)
    t, tt, ttt = elecrate_manager.print_aggregated_bill(bill)
    print(t)

    # 2) Get the electricity price per type of metric, for a specific period

    tariff_openei_jsondata = OpenEI_tariff(utility_id='14328', sector='Commercial', tariff_rate_of_interest='B-19S')

    if tariff_openei_jsondata.read_from_json(filename="tariff_revised/u14328_Commercial_B19S_revised.json") == 0:
        print("Tariff read from JSON successful")
    else:
        print("An error occurred when reading the JSON file")
        exit()

    elecrate_manager = ElectricityRateManager()

    tariff_struct_from_openei_data(tariff_openei_jsondata, elecrate_manager)  
    # This analyses the raw data from the openEI request and populate the "CostCalculator" object


    start_date_sig= datetime(2020, 1, 1, hour=0, minute=0, second=0)
    end_date_sig = datetime(2020, 1, 7, hour=23, minute=59, second=59)
    time_range = (start_date_sig,end_date_sig)
    
    timestep = TariffElemPeriod.QUARTERLY  # We want a 1h period

    price_elec, map = elecrate_manager.get_electricity_price(time_range, timestep)
    #price_elec, map_columns = aelec_rate_handler.get_electricity_price(time_range, timestep)
    
    #`price_elec`is a pandas` dataframe containing the timeseries data, 
    #and `map_columns` is a dictionnary mapping the columns of `price_elec` to a type of rate.
    
    


--- Loading meter data ...
--- Calling OpenEI API ...
Calculating the bill for the period 2017-08-01 00:00:00 to 2017-08-30 23:59:59
time_label:
 2017-08 label:
 customer_fix_charge bill_ddata:
 (30, 4.536581917808219)
mult_energy_unit: 1000.0
mult_cost_unit: 1.0
hourly_rate: 0.0786
df_values_in_hour: 5704.0
daily_rate: [0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.1482, 0.1482, 0.1482, 0.1482, 0.3223, 0.3223, 0.3223, 0.3223, 0.3223, 0.3223, 0.1482, 0.1482, 0.1482, 0.0786, 0.0786, 0.0786]
rate is just TOU 5.704 0.4483344
hourly_rate: 0.0786
df_values_in_hour: 5840.0
daily_rate: [0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.1482, 0.1482, 0.1482, 0.1482, 0.3223, 0.3223, 0.3223, 0.3223, 0.3223, 0.3223, 0.1482, 0.1482, 0.1482, 0.0786, 0.0786, 0.0786]
rate is just TOU 11.544 0.9073584
hourly_rate: 0.0786
df_values_in_hour: 5744.0
daily_rate: [0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.1482, 0.1482, 0.1482, 0.1482, 0.3223, 0.32

rate is just TOU 969.8960000000004 113.58917120000002
hourly_rate: 0.0786
df_values_in_hour: 6736.0
daily_rate: [0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.1482, 0.1482, 0.1482, 0.1482, 0.3223, 0.3223, 0.3223, 0.3223, 0.3223, 0.3223, 0.1482, 0.1482, 0.1482, 0.0786, 0.0786, 0.0786]
rate is just TOU 976.6320000000004 114.11862080000003
hourly_rate: 0.0786
df_values_in_hour: 6232.0
daily_rate: [0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.1482, 0.1482, 0.1482, 0.1482, 0.3223, 0.3223, 0.3223, 0.3223, 0.3223, 0.3223, 0.1482, 0.1482, 0.1482, 0.0786, 0.0786, 0.0786]
rate is just TOU 982.8640000000004 114.60845600000003
hourly_rate: 0.0786
df_values_in_hour: 5600.0
daily_rate: [0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.1482, 0.1482, 0.1482, 0.1482, 0.3223, 0.3223, 0.3223, 0.3223, 0.3223, 0.3223, 0.1482, 0.1482, 0.1482, 0.0786, 0.0786, 0.0786]
rate is just TOU 988.4640000000004 115.04861600000004
mult_energy_unit: 1000.0
mult_c

rate is just TOU 1833.5919999999999 202.96027760000004
hourly_rate: 0.0786
df_values_in_hour: 6792.0
daily_rate: [0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.1482, 0.1482, 0.1482, 0.1482, 0.3223, 0.3223, 0.3223, 0.3223, 0.3223, 0.3223, 0.1482, 0.1482, 0.1482, 0.0786, 0.0786, 0.0786]
rate is just TOU 1840.3839999999998 203.49412880000003
mult_energy_unit: 1000.0
mult_cost_unit: 1.0
hourly_rate: 0.0786
df_values_in_hour: 6288.0
daily_rate: [0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786]
rate is just TOU 1846.6719999999998 203.98836560000004
hourly_rate: 0.0786
df_values_in_hour: 6032.0
daily_rate: [0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786]
rate is just TOU 1852.7039999999997 2

df_values_in_hour: 4672.0
daily_rate: [0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.1482, 0.1482, 0.1482, 0.1482, 0.3223, 0.3223, 0.3223, 0.3223, 0.3223, 0.3223, 0.1482, 0.1482, 0.1482, 0.0786, 0.0786, 0.0786]
rate is just TOU 3003.9759999999965 320.5080312
hourly_rate: 0.1482
df_values_in_hour: 3104.0
daily_rate: [0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.1482, 0.1482, 0.1482, 0.1482, 0.3223, 0.3223, 0.3223, 0.3223, 0.3223, 0.3223, 0.1482, 0.1482, 0.1482, 0.0786, 0.0786, 0.0786]
rate is just TOU 3007.0799999999963 320.968044
hourly_rate: 0.1482
df_values_in_hour: 392.0
daily_rate: [0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.1482, 0.1482, 0.1482, 0.1482, 0.3223, 0.3223, 0.3223, 0.3223, 0.3223, 0.3223, 0.1482, 0.1482, 0.1482, 0.0786, 0.0786, 0.0786]
rate is just TOU 3007.471999999996 321.02613840000004
hourly_rate: 0.1482
df_values_in_hour: 120.0
daily_rate: [0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.0786, 0.078

In [6]:
price_elec

,customer_energy_charge,customer_demand_charge_season,customer_demand_charge_tou,pdp_event_energy_charge,pdp_non_event_energy_credit,pdp_non_event_demand_credit
2020-01-01 00:00:00,0.11567,8.91,2.46,NaN,NaN,NaN
2020-01-01 00:15:00,0.11567,8.91,2.46,NaN,NaN,NaN
2020-01-01 00:30:00,0.11567,8.91,2.46,NaN,NaN,NaN
2020-01-01 00:45:00,0.11567,8.91,2.46,NaN,NaN,NaN
2020-01-01 01:00:00,0.11567,8.91,2.46,NaN,NaN,NaN
...,...,...,...,...,...,...
2020-01-07 22:45:00,0.11567,8.91,2.46,NaN,NaN,NaN
2020-01-07 23:00:00,0.11567,8.91,2.46,NaN,NaN,NaN
2020-01-07 23:15:00,0.11567,8.91,2.46,NaN,NaN,NaN
2020-01-07 23:30:00,0.11567,8.91,2.46,NaN,NaN,NaN


In [7]:
for block_rate in tariff_openei_jsondata.data_openei:
    print(block_rate['enddate'])
    print(block_rate['energyratestructure'])
    tariff_dates = (block_rate['startdate'],block_rate['enddate'])
    tariff_energy_obj = get_energyrate_obj_from_openei(block_rate)#TOURateSchedule
    print(len(tariff_energy_obj.get_rate(1,3)))

#     df = 
#     data_col = 
#     TouEnergyChargeTariff.compute_monthly_bill(df, data_col)

2020-12-31 23:59:59+00:00
[[{'rate': 0.07985, 'unit': 'kWh'}], [{'rate': 0.11567, 'unit': 'kWh'}], [{'rate': 0.15798, 'unit': 'kWh'}], [{'rate': 0.11914, 'unit': 'kWh'}], [{'rate': 0.1796, 'unit': 'kWh'}], [{'rate': 0.36344, 'unit': 'kWh'}]]
24


# TOU tier tests

In [8]:
#TOU - tier tests
__author__ = 'Serena Patel'


# ----------- TEST DEMO -------------- #
READ_FROM_JSON = False


if __name__ == '__main__':

    meter_uuid = 'e9c51ce5-4aa1-399c-8172-92073e273a0b'

    #
    ### Reading power consumption data
    #

    print("--- Loading meter data ...")
    df = pd.read_csv('meter.csv', index_col=0)  # import times series energy data for meters
    df.index.name = 'Time'
    df = df.set_index(pd.to_datetime(df.index, infer_datetime_format=True, utc=True))
    df["date"] = df.index

    data_meter = df[meter_uuid]
    data_meter = utc_to_local(data_meter, local_zone="America/Los_Angeles")
    
    #change to 2020 tests
    data_meter_index= pd.date_range(start = '1/1/2018', end = '5/13/2021', freq = 'H', closed = 'right')
    data_meter.index = data_meter_index
    #
    ### Reading OpenEI-based tariff rates, and binding it to the ElectricityRateManager
    #

    print("--- Calling OpenEI API ...")

    # (a) Example of using the OpenEI WEB API to populate tariff_data
    tou_tier_name = 'E-TOU-C3 Residential Time of Use Baseline Region T'
    #tou_tier_name = 'E-TOU-C3'
#https://openei.org/apps/USURDB/rate/view/5d711d0a5457a3b72c466f68#3__Energy
    tariff_openei_apidata = OpenEI_tariff(utility_id='14328',
                                       sector='Residential',
                                       tariff_rate_of_interest=tou_tier_name,
                                       distrib_level_of_interest=None,
                                       # it is at the secondary level, so not specified in the name
                                       phasewing=None,
                                       # the word 'Poly' is to be excluded, because the names may omit this info ..
                                       tou=True,
                                       option_exclusion=['(X)', '(W)', 'Poly']) 
    
    tariff_openei_apidata.call_api(store_as_json=True)
        #tariff data
        
    # (b) Example of reading local tariff data, binding to elec_rate_handler
    #encoded according to OpenEI structure


    elecrate_manager = ElectricityRateManager() #elec_rate_handler

    # Binding an instance of ElectricityRateManager to a specific OpenEI tariff

    tariff_struct_from_openei_data(tariff_openei_apidata, elecrate_manager)  # This analyses the raw data from the openEI request and populate the "CostCalculator" object

    # BILLING PERIOD
    start_date_bill = datetime(2020, 8, 1, hour=0, minute=0, second=0)
    end_date_bill = datetime(2020, 8, 30, hour=23, minute=59, second=59)
    mask = (data_meter.index >= start_date_bill) & (data_meter.index <= end_date_bill)
    data_meter = data_meter.loc[mask]
    data_meter = data_meter.fillna(0)

    # 1) Get the bill over the period

    print("Calculating the bill for the period {0} to {1}".format(start_date_bill, end_date_bill))
    bill = elecrate_manager.compute_bill(data_meter, monthly_detailed=True)
    t, tt, ttt = elecrate_manager.print_aggregated_bill(bill)
    print(bill)

#     # 2) Get the electricity price per type of metric, for a specific period
#     tariff_openei_jsondata = OpenEI_tariff(utility_id='14328', sector='Residential', tariff_rate_of_interest=tou_tier_name)

#     if tariff_openei_jsondata.read_from_json(filename="electricitycostcalculator/openei_tariff/cache_api/u14328_Residential_E-TOU-C3 Residential Time of Use Baseline Region T_TOU.json") == 0:
#         print("Tariff read from JSON successful")
#     else:
#         print("An error occurred when reading the JSON file")
#         exit()
        
#     elecrate_manager = ElectricityRateManager()

#     tariff_struct_from_openei_data(tariff_openei_jsondata, elecrate_manager)  
#     # This analyses the raw data from the openEI request and populate the "CostCalculator" object



#     start_date_sig= datetime(2020, 1, 1, hour=0, minute=0, second=0)
#     end_date_sig = datetime(2020, 1, 7, hour=23, minute=59, second=59)
#     time_range = (start_date_sig,end_date_sig)
    
#     timestep = TariffElemPeriod.QUARTERLY  # We want a 1h period

#     price_elec, map = elecrate_manager.get_electricity_price(time_range, timestep)
#     print(price_elec)
#     #price_elec, map_columns = aelec_rate_handler.get_electricity_price(time_range, timestep)
    
#     #`price_elec`is a pandas` dataframe containing the timeseries data, 
#     #and `map_columns` is a dictionnary mapping the columns of `price_elec` to a type of rate.
    
    


--- Loading meter data ...
--- Calling OpenEI API ...
tou-tier
Calculating the bill for the period 2020-08-01 00:00:00 to 2020-08-30 23:59:59
time_label:
 2020-08 label:
 customer_fix_charge bill_ddata:
 (30, 9.863013698630137)
mult_energy_unit: 1000.0
mult_cost_unit: 1.0
hourly_rate: (0.23882, 6.8, 0.32068)
df_values_in_hour: 5704.0
daily_rate: [(0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.30226, 6.8, 0.38412), (0.30226, 6.8, 0.38412), (0.30226, 6.8, 0.38412), (0.30226, 6.8, 0.38412), (0.30226, 6.8, 0.38412), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068)]
rate in first tier 5.704 1.36222928
hourly_rate: (0

rate in second tier 1134.5520000000006 364.8861777600003
hourly_rate: (0.23882, 6.8, 0.32068)
df_values_in_hour: 2544.0
daily_rate: [(0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.30226, 6.8, 0.38412), (0.30226, 6.8, 0.38412), (0.30226, 6.8, 0.38412), (0.30226, 6.8, 0.38412), (0.30226, 6.8, 0.38412), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068)]
rate in second tier 1137.0960000000007 365.7019876800003
hourly_rate: (0.23882, 6.8, 0.32068)
df_values_in_hour: 1024.0
daily_rate: [(0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882,

rate in second tier 1975.3199999999997 647.4515102400006
hourly_rate: (0.23882, 6.8, 0.32068)
df_values_in_hour: 0.0
daily_rate: [(0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.30226, 6.8, 0.38412), (0.30226, 6.8, 0.38412), (0.30226, 6.8, 0.38412), (0.30226, 6.8, 0.38412), (0.30226, 6.8, 0.38412), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068)]
rate in second tier 1975.3199999999997 647.4515102400006
hourly_rate: (0.23882, 6.8, 0.32068)
df_values_in_hour: 64.0
daily_rate: [(0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8,

In [9]:
data_meter

2020-08-01 00:00:00    5704.0
2020-08-01 01:00:00    5840.0
2020-08-01 02:00:00    5744.0
2020-08-01 03:00:00    5408.0
2020-08-01 04:00:00    5272.0
                        ...  
2020-08-30 19:00:00    6632.0
2020-08-30 20:00:00    7568.0
2020-08-30 21:00:00    7064.0
2020-08-30 22:00:00    7288.0
2020-08-30 23:00:00    6680.0
Freq: H, Name: e9c51ce5-4aa1-399c-8172-92073e273a0b, Length: 720, dtype: float64

In [10]:
bill = elecrate_manager.compute_bill(data_meter, monthly_detailed=True)
bill

time_label:
 2020-08 label:
 customer_fix_charge bill_ddata:
 (30, 9.863013698630137)
mult_energy_unit: 1000.0
mult_cost_unit: 1.0
hourly_rate: (0.23882, 6.8, 0.32068)
df_values_in_hour: 5704.0
daily_rate: [(0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.30226, 6.8, 0.38412), (0.30226, 6.8, 0.38412), (0.30226, 6.8, 0.38412), (0.30226, 6.8, 0.38412), (0.30226, 6.8, 0.38412), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068)]
rate in first tier 5.704 1.36222928
hourly_rate: (0.23882, 6.8, 0.32068)
df_values_in_hour: 5840.0
daily_rate: [(0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.238

daily_rate: [(0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.30226, 6.8, 0.38412), (0.30226, 6.8, 0.38412), (0.30226, 6.8, 0.38412), (0.30226, 6.8, 0.38412), (0.30226, 6.8, 0.38412), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068)]
rate in second tier 1045.8160000000005 335.3477771200003
hourly_rate: (0.30226, 6.8, 0.38412)
df_values_in_hour: 4896.0
daily_rate: [(0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.2

rate in second tier 2134.5679999999993 701.6251812800006
hourly_rate: (0.23882, 6.8, 0.32068)
df_values_in_hour: 6008.0
daily_rate: [(0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.30226, 6.8, 0.38412), (0.30226, 6.8, 0.38412), (0.30226, 6.8, 0.38412), (0.30226, 6.8, 0.38412), (0.30226, 6.8, 0.38412), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068)]
rate in second tier 2140.575999999999 703.5518267200006
hourly_rate: (0.23882, 6.8, 0.32068)
df_values_in_hour: 5752.0
daily_rate: [(0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 6.8, 0.32068), (0.23882, 

{'2020-08': {'customer_fix_charge': (30, 9.863013698630137),
  'customer_energy_charge': (3057.7999999999956, 1014.2399838400019),
  'customer_demand_charge_season': {},
  'customer_demand_charge_tou': {},
  'pdp_event_energy_charge': (0, 0),
  'pdp_non_event_energy_credit': (0, 0),
  'pdp_non_event_demand_credit': {}}}

In [27]:
tou_tier_name

'E-TOU-C3 Residential Time of Use Baseline Region T'

In [29]:
tariff_openei_jsondata = OpenEI_tariff(utility_id='14328', sector='Residential', tariff_rate_of_interest=tou_tier_name)
if tariff_openei_jsondata.read_from_json(filename="electricitycostcalculator/openei_tariff/cache_api/u14328_Residential_E-TOU-C3 Residential Time of Use Baseline Region T_TOU.json") == 0:
    print("Tariff read from JSON successful")
else:
    print("An error occurred when reading the JSON file")
    exit()
        
for block_rate in tariff_openei_jsondata.data_openei:
    print(block_rate['startdate'])
    print(block_rate['enddate'])
#     print(block_rate['energyratestructure'])
#     x = get_energyrate_obj_from_openei(block_rate)
#     print(x.main_structure)
    tariff = get_block_obj_from_openei(block_rate)
    #print(tariff.main_structure)
    rate_struct = tariff.get_daily_rate(1/1/2020)
    print(tariff.get_rate_in_day(rate_struct, (5,5)))
    #x = tariff.get_daily_rate(1/1/2020)[0][0]
    #print(x, isinstance(x, float))
    #print('max' in block_rate['energyratestructure'][0][0].keys())
#     tariff_energy_raw_struct = get_raw_blocks_struct(block_rate)
    #print(tariff_energy_raw_struct[0][0])
    #print(x.get_rate(6,5))
   # print(x.main_structure)


    #c
# period = 0
# rate_map = tariff_energy_raw_struct[0]
# rate_map[period][0]

Tariff read from JSON successful
2019-07-01 07:00:00+00:00
2021-03-18 02:26:51+00:00
(0.19784, 8.2, 0.2797)


In [13]:
block_tariff_struct_from_openei_data(tariff_openei_jsondata, elecrate_manager)

'0rates'

next steps --> apply the billing style you were using before


# Check TOU tier rate


Limitations
* by month (days in the time period) 
* limited to tou-tier with only 2 tiers in each tou period --> need to change to accomadate multiple tiers within the TOU periods 

In [14]:
for block_rate in tariff_openei_apidata.data_openei:
    tariff = get_raw_blocks_struct(block_rate)  # This analyses the raw data from the openEI request and populate the "CostCalculator" object
    

In [15]:
#format the data_meter
df_meter = pd.DataFrame(data_meter)
df_meter['Cummulative Sum (Wh)'] = np.cumsum(df_meter)
df_meter['month'] = df_meter.index.month #1 to 12
df_meter['weekday']= df_meter.index.weekday #0 to 6
df_meter['hour'] = df_meter.index.hour #0 to 23
df_meter['year'] = df_meter.index.year
df_meter['bill'] = np.zeros(len(df_meter))


# start_date_bill = datetime(2017, 1, 1, hour=0, minute=0, second=0)
# end_date_bill = datetime(2017, 2, 1, hour=23, minute=59, second=59)
# mask = (data_meter.index >= start_date_bill) & (data_meter.index <= end_date_bill)
# data_meter = data_meter.loc[mask]
# data_meter = data_meter.fillna(0)

#make the period column
period_map = np.array([])
for i in np.arange(len(df_meter)):
    month = df_meter['month'][i]-1 #index month
    hour = df_meter['hour'][i] #index hour
    if df_meter['weekday'][i] == 5 or df_meter['weekday'][i] == 6: #weekend
        period = tariff['0weekends'][month][hour] 
        period_map = np.append(period_map,period)

    else: #weekday
        period = tariff['0weekdays'][month][hour] 
        period_map = np.append(period_map,period)


df_meter['period'] = period_map

days_in_month=[np.array(monthrange(2020, i))[1] for i in np.arange(1,13)] 

def billing(df_row,days_in_month,tariff):
#index by period
    period = int(df_row['period'])  
    
    

    month = int(df_row['month'])
    year = int(df_row['year'])
    days = int(days_in_month[month-1]) #number of days in the month to determine the month's threshold values
    cumsum = float(df_row['Cummulative Sum (Wh)']/1000) #kWh cummulative sum 
    energy = float(df_row['e9c51ce5-4aa1-399c-8172-92073e273a0b'])/1000 

    #number of tiers --> index is the tier 
    thresh0_0 = float(tariff['0rates'][period][0]['max'])*days #kWh/month
    rate0_0 = float(tariff['0rates'][period][0]['rate']) #$/kWh
    rate0_1 = float(tariff['0rates'][period][1]['rate']) #$/kWh


    bill = 0
    
    if cumsum <= thresh0_0: 
        bill = energy*rate0_0
        print('tier 1', cumsum, bill, rate0_0, thresh0_0)

    if cumsum > thresh0_0: 
        bill = energy*rate0_1
        print('tier 2', cumsum, bill, rate0_1)


    return bill

In [16]:
df_meter

,e9c51ce5-4aa1-399c-8172-92073e273a0b,Cummulative Sum (Wh),month,weekday,hour,year,bill,period
2020-08-01 00:00:00,5704.0,5704.0,8,5,0,2020,0.0,1.0
2020-08-01 01:00:00,5840.0,11544.0,8,5,1,2020,0.0,1.0
2020-08-01 02:00:00,5744.0,17288.0,8,5,2,2020,0.0,1.0
2020-08-01 03:00:00,5408.0,22696.0,8,5,3,2020,0.0,1.0
2020-08-01 04:00:00,5272.0,27968.0,8,5,4,2020,0.0,1.0
...,...,...,...,...,...,...,...,...
2020-08-30 19:00:00,6632.0,3029200.0,8,6,19,2020,0.0,0.0
2020-08-30 20:00:00,7568.0,3036768.0,8,6,20,2020,0.0,0.0
2020-08-30 21:00:00,7064.0,3043832.0,8,6,21,2020,0.0,1.0
2020-08-30 22:00:00,7288.0,3051120.0,8,6,22,2020,0.0,1.0


In [17]:
billing = [billing(df_meter.loc[i,:], days_in_month,tariff) for i in df_meter.index]

tier 1 5.704 1.36222928 0.23882 210.79999999999998
tier 1 11.544 1.3947088 0.23882 210.79999999999998
tier 1 17.288 1.37178208 0.23882 210.79999999999998
tier 1 22.696 1.29153856 0.23882 210.79999999999998
tier 1 27.968 1.2590590400000001 0.23882 210.79999999999998
tier 1 33.312 1.2762540800000002 0.23882 210.79999999999998
tier 1 37.592 1.0221496 0.23882 210.79999999999998
tier 1 40.144 0.60946864 0.23882 210.79999999999998
tier 1 41.848 0.40694928 0.23882 210.79999999999998
tier 1 41.904 0.013373920000000001 0.23882 210.79999999999998
tier 1 41.912 0.0019105600000000002 0.23882 210.79999999999998
tier 1 41.912 0.0 0.23882 210.79999999999998
tier 1 41.912 0.0 0.23882 210.79999999999998
tier 1 42.008 0.02292672 0.23882 210.79999999999998
tier 1 42.016 0.0019105600000000002 0.23882 210.79999999999998
tier 1 42.68 0.15857648000000002 0.23882 210.79999999999998
tier 1 44.808 0.64320928 0.30226 210.79999999999998
tier 1 52.288 2.2609048 0.30226 210.79999999999998
tier 1 59.392 2.1472550399

tier 2 2043.488 1.7829808 0.32068
tier 2 2049.352 1.88046752 0.32068
tier 2 2055.096 1.84198592 0.32068
tier 2 2060.472 1.7239756800000001 0.32068
tier 2 2065.272 1.539264 0.32068
tier 2 2068.952 1.1801024000000002 0.32068
tier 2 2070.864 0.61314016 0.32068
tier 2 2074.208 1.07235392 0.32068
tier 2 2075.456 0.40020864 0.32068
tier 2 2075.52 0.020523520000000003 0.32068
tier 2 2075.616 0.03078528 0.32068
tier 2 2075.64 0.00769632 0.32068
tier 2 2075.744 0.03335072 0.32068
tier 2 2077.448 0.65454048 0.38412
tier 2 2081.552 1.5764284800000001 0.38412
tier 2 2086.944 2.0711750400000004 0.38412
tier 2 2093.064 2.3508144 0.38412
tier 2 2100.744 2.9500416 0.38412
tier 2 2108.632 2.52952384 0.32068
tier 2 2115.32 2.14470784 0.32068
tier 2 2122.296 2.2370636800000003 0.32068
tier 2 2128.456 1.9753888000000002 0.32068
tier 2 2134.568 1.9599961600000002 0.32068
tier 2 2140.576 1.9266454400000002 0.32068
tier 2 2146.328 1.84455136 0.32068
tier 2 2151.912 1.79067712 0.32068
tier 2 2157.056 1.649577

In [18]:
sum(billing) #very close, off by $1-2 

1013.4606766400018

## Tests for update to TOU-C

In [19]:
#check if TOU-C3 matches the tariff sheet
print('summer peak tier 1', 0.1765+0.11147-0.03606+0.03595+0.00314-0.00066+0.01240+0.00088+0.00110-0.00057+0.0058+0.00217)

summer peak tier 1 0.31211999999999995


"Adjustments included in 
Rate = Generation + Distribution + Transmission + Transmission Rate Adjustments + Reliability Services + Public Purpose Programs + Nuclear Decommissioning + Competition Transition Charges + Energy Cost Recovery Amount + DWR Bond + New System Generation Charge

In [35]:
#summer peak 
#baseline
baseline = 0.41813
baseline_credit = 0.07584
print('summer peak ', 'tier 1:',baseline-baseline_credit,'tier 2:', baseline)

#summer offpeak
baseline = 0.35469
baseline_credit = 0.07584
print('summer off peak ', 'tier 1:',baseline-baseline_credit,'tier 2:', baseline)



#winter peak

baseline = 0.32104
baseline_credit = 0.07584
print('winter peak ', 'tier 1:',baseline-baseline_credit,'tier 2:', baseline)


#winter offpeak
baseline = 0.30372
baseline_credit = 0.07584
print('winter off peak ', 'tier 1:',baseline-baseline_credit,'tier 2:', baseline)


#delivery min bill amount
delivery = 0.32854 #$/meter/day

#california climate credit applied in April and October
ccc = 17.20

# generation = 0.17650
# distribution = 0.11147

# #other charges
# transmission = 0.03595
# transmission_adj = 0.00314
# reliability = 0.00066
# ppp=0.01240
# nuc_decom = 0.00088
# comptc = 0.00110
# ecra = 0.00057
# dwr = 0.0058
# nsgc = 0.00217

print('all-electric summer tier 1:', 7.5)
print('all-electric winter tier 1:', 13.6)
print('regular summer tier 1:', 6.8)
print('regular winter tier 1:', 8.2)

# generation + distribution + transmission + transmission_adj - reliability + ppp+nuc_decom+comptc-ecra+dwr+nsgc-0.03606

summer peak  tier 1: 0.34229 tier 2: 0.41813
summer off peak  tier 1: 0.27885 tier 2: 0.35469
winter peak  tier 1: 0.24519999999999997 tier 2: 0.32104
winter off peak  tier 1: 0.22787999999999997 tier 2: 0.30372
all-electric summer tier 1: 7.5
all-electric winter tier 1: 13.6
regular summer tier 1: 6.8
regular winter tier 1: 8.2


In [33]:

#json_name ='tariff_revised/u14328_Residential_E-TOU-C_TOU_Basic_Region_T_revised.json'
json_name = 'tariff_revised/TOU-C.json'
tariff_openei_jsondata.read_from_json(filename=json_name)


0

In [34]:
#TOU - tier tests
__author__ = 'Serena Patel'


# ----------- TEST DEMO -------------- #
READ_FROM_JSON = True


if __name__ == '__main__':

    meter_uuid = 'e9c51ce5-4aa1-399c-8172-92073e273a0b'

    #
    ### Reading power consumption data
    #

    print("--- Loading meter data ...")
    df = pd.read_csv('meter.csv', index_col=0)  # import times series energy data for meters
    df.index.name = 'Time'
    df = df.set_index(pd.to_datetime(df.index, infer_datetime_format=True, utc=True))
    df["date"] = df.index

    data_meter = df[meter_uuid]
    data_meter = utc_to_local(data_meter, local_zone="America/Los_Angeles")
    
    #change to 2020 tests
    data_meter_index= pd.date_range(start = '1/1/2018', end = '5/13/2021', freq = 'H', closed = 'right')
    data_meter.index = data_meter_index
    #
    ### Reading OpenEI-based tariff rates, and binding it to the ElectricityRateManager
    #

    print("--- Calling OpenEI API ...")

#     # (a) Example of using the OpenEI WEB API to populate tariff_data
#     tou_tier_name = 'E-TOU-C3 Residential Time of Use Baseline Region T'
#     #tou_tier_name = 'E-TOU-C3'
# #https://openei.org/apps/USURDB/rate/view/5d711d0a5457a3b72c466f68#3__Energy
#     tariff_openei_apidata = OpenEI_tariff(utility_id='14328',
#                                        sector='Residential',
#                                        tariff_rate_of_interest=tou_tier_name,
#                                        distrib_level_of_interest=None,
#                                        # it is at the secondary level, so not specified in the name
#                                        phasewing=None,
#                                        # the word 'Poly' is to be excluded, because the names may omit this info ..
#                                        tou=True,
#                                        option_exclusion=['(X)', '(W)', 'Poly']) 
    
#     tariff_openei_apidata.call_api(store_as_json=True)
        #tariff data
        
    # (b) Example of reading local tariff data, binding to elec_rate_handler
    #encoded according to OpenEI structure


    elecrate_manager = ElectricityRateManager() #elec_rate_handler

    # Binding an instance of ElectricityRateManager to a specific OpenEI tariff

    tariff_struct_from_openei_data(tariff_openei_jsondata, elecrate_manager)  # This analyses the raw data from the openEI request and populate the "CostCalculator" object

    # BILLING PERIOD
    start_date_bill = datetime(2020, 8, 1, hour=0, minute=0, second=0)
    end_date_bill = datetime(2020, 8, 30, hour=23, minute=59, second=59)
    mask = (data_meter.index >= start_date_bill) & (data_meter.index <= end_date_bill)
    data_meter = data_meter.loc[mask]
    data_meter = data_meter.fillna(0)

    # 1) Get the bill over the period

    print("Calculating the bill for the period {0} to {1}".format(start_date_bill, end_date_bill))
    bill = elecrate_manager.compute_bill(data_meter, monthly_detailed=True)
    t, tt, ttt = elecrate_manager.print_aggregated_bill(bill)
    print(bill)

#     # 2) Get the electricity price per type of metric, for a specific period
#     tariff_openei_jsondata = OpenEI_tariff(utility_id='14328', sector='Residential', tariff_rate_of_interest=tou_tier_name)

#     if tariff_openei_jsondata.read_from_json(filename="electricitycostcalculator/openei_tariff/cache_api/u14328_Residential_E-TOU-C3 Residential Time of Use Baseline Region T_TOU.json") == 0:
#         print("Tariff read from JSON successful")
#     else:
#         print("An error occurred when reading the JSON file")
#         exit()
        
#     elecrate_manager = ElectricityRateManager()

#     tariff_struct_from_openei_data(tariff_openei_jsondata, elecrate_manager)  
#     # This analyses the raw data from the openEI request and populate the "CostCalculator" object



#     start_date_sig= datetime(2020, 1, 1, hour=0, minute=0, second=0)
#     end_date_sig = datetime(2020, 1, 7, hour=23, minute=59, second=59)
#     time_range = (start_date_sig,end_date_sig)
    
#     timestep = TariffElemPeriod.QUARTERLY  # We want a 1h period

#     price_elec, map = elecrate_manager.get_electricity_price(time_range, timestep)
#     print(price_elec)
#     #price_elec, map_columns = aelec_rate_handler.get_electricity_price(time_range, timestep)
    
#     #`price_elec`is a pandas` dataframe containing the timeseries data, 
#     #and `map_columns` is a dictionnary mapping the columns of `price_elec` to a type of rate.
    
    


--- Loading meter data ...
--- Calling OpenEI API ...
tou-tier
Calculating the bill for the period 2020-08-01 00:00:00 to 2020-08-30 23:59:59
time_label:
 2020-08 label:
 customer_fix_charge bill_ddata:
 (30, 9.863013698630137)
mult_energy_unit: 1000.0
mult_cost_unit: 1.0
hourly_rate: (0.27885, 6.8, 0.35469)
df_values_in_hour: 5704.0
daily_rate: [(0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.342229, 6.8, 0.41813), (0.342229, 6.8, 0.41813), (0.342229, 6.8, 0.41813), (0.342229, 6.8, 0.41813), (0.342229, 6.8, 0.41813), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469)]
rate in first tier 5.704 1.5905603999999998
ho

daily_rate: [(0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.342229, 6.8, 0.41813), (0.342229, 6.8, 0.41813), (0.342229, 6.8, 0.41813), (0.342229, 6.8, 0.41813), (0.342229, 6.8, 0.41813), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469)]
rate in second tier 1082.5920000000003 386.22773865600027
mult_energy_unit: 1000.0
mult_cost_unit: 1.0
hourly_rate: (0.27885, 6.8, 0.35469)
df_values_in_hour: 6544.0
daily_rate: [(0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.

df_values_in_hour: 6552.0
daily_rate: [(0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.342229, 6.8, 0.41813), (0.342229, 6.8, 0.41813), (0.342229, 6.8, 0.41813), (0.342229, 6.8, 0.41813), (0.342229, 6.8, 0.41813), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469)]
rate in second tier 1939.4559999999997 703.0966810559999
hourly_rate: (0.27885, 6.8, 0.35469)
df_values_in_hour: 5936.0
daily_rate: [(0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469)

daily_rate: [(0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.342229, 6.8, 0.41813), (0.342229, 6.8, 0.41813), (0.342229, 6.8, 0.41813), (0.342229, 6.8, 0.41813), (0.342229, 6.8, 0.41813), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469)]
rate in second tier 2869.191999999996 1048.6562276959992
hourly_rate: (0.27885, 6.8, 0.35469)
df_values_in_hour: 5576.0
daily_rate: [(0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469), (0.27885, 6.8, 0.35469),

### check the hand model

In [23]:
# check the hand model
# tariff_openei_jsondata.read_from_json(filename=json_name)
# tariff_struct_from_openei_data(tariff_openei_jsondata, elecrate_manager)  # This analyses the raw data from the openEI request and populate the "CostCalculator" object


# tariff = get_raw_blocks_struct(tariff_openei_jsondata)  # This analyses the raw data from the openEI request and populate the "CostCalculator" object

tariff = get_raw_blocks_struct(tariff_openei_jsondata.data_openei[0])


In [24]:
## tariff
df_meter = pd.DataFrame(data_meter)
df_meter['Cummulative Sum (Wh)'] = np.cumsum(df_meter)
df_meter['month'] = df_meter.index.month #1 to 12
df_meter['weekday']= df_meter.index.weekday #0 to 6
df_meter['hour'] = df_meter.index.hour #0 to 23
df_meter['year'] = df_meter.index.year
df_meter['bill'] = np.zeros(len(df_meter))


# start_date_bill = datetime(2017, 1, 1, hour=0, minute=0, second=0)
# end_date_bill = datetime(2017, 2, 1, hour=23, minute=59, second=59)
# mask = (data_meter.index >= start_date_bill) & (data_meter.index <= end_date_bill)
# data_meter = data_meter.loc[mask]
# data_meter = data_meter.fillna(0)

#make the period column
period_map = np.array([])
for i in np.arange(len(df_meter)):
    month = df_meter['month'][i]-1 #index month
    hour = df_meter['hour'][i] #index hour
    if df_meter['weekday'][i] == 5 or df_meter['weekday'][i] == 6: #weekend
        period = tariff['0weekends'][month][hour] 
        period_map = np.append(period_map,period)

    else: #weekday
        period = tariff['0weekdays'][month][hour] 
        period_map = np.append(period_map,period)


df_meter['period'] = period_map

days_in_month=[np.array(monthrange(2020, i))[1] for i in np.arange(1,13)] 

def billing(df_row,days_in_month,tariff):
#index by period
    period = int(df_row['period'])  
    
    

    month = int(df_row['month'])
    year = int(df_row['year'])
    days = int(days_in_month[month-1]) #number of days in the month to determine the month's threshold values
    cumsum = float(df_row['Cummulative Sum (Wh)']/1000) #kWh cummulative sum 
    energy = float(df_row['e9c51ce5-4aa1-399c-8172-92073e273a0b'])/1000 

    #number of tiers --> index is the tier 
    thresh0_0 = float(tariff['0rates'][period][0]['max'])*days #kWh/month
    rate0_0 = float(tariff['0rates'][period][0]['rate']) #$/kWh
    rate0_1 = float(tariff['0rates'][period][1]['rate']) #$/kWh


    bill = 0
    
    if cumsum <= thresh0_0: 
        bill = energy*rate0_0
        print('tier 1', cumsum, bill, rate0_0, thresh0_0)

    if cumsum > thresh0_0: 
        bill = energy*rate0_1
        print('tier 2', cumsum, bill, rate0_1)


    return bill

In [25]:
billing = [billing(df_meter.loc[i,:], days_in_month,tariff) for i in df_meter.index]

tier 1 5.704 1.5905603999999998 0.27885 210.79999999999998
tier 1 11.544 1.6284839999999998 0.27885 210.79999999999998
tier 1 17.288 1.6017143999999999 0.27885 210.79999999999998
tier 1 22.696 1.5080208 0.27885 210.79999999999998
tier 1 27.968 1.4700972 0.27885 210.79999999999998
tier 1 33.312 1.4901744000000001 0.27885 210.79999999999998
tier 1 37.592 1.193478 0.27885 210.79999999999998
tier 1 40.144 0.7116252 0.27885 210.79999999999998
tier 1 41.848 0.4751604 0.27885 210.79999999999998
tier 1 41.904 0.0156156 0.27885 210.79999999999998
tier 1 41.912 0.0022308 0.27885 210.79999999999998
tier 1 41.912 0.0 0.27885 210.79999999999998
tier 1 41.912 0.0 0.27885 210.79999999999998
tier 1 42.008 0.026769599999999998 0.27885 210.79999999999998
tier 1 42.016 0.0022308 0.27885 210.79999999999998
tier 1 42.68 0.1851564 0.27885 210.79999999999998
tier 1 44.808 0.728263312 0.342229 210.79999999999998
tier 1 52.288 2.55987292 0.342229 210.79999999999998
tier 1 59.392 2.431194816 0.342229 210.799999

tier 2 1889.744 0.20404744 0.41813
tier 2 1892.28 1.06037768 0.41813
tier 2 1896.16 1.6223444 0.41813
tier 2 1900.744 1.91670792 0.41813
tier 2 1907.2 2.6994472800000002 0.41813
tier 2 1913.368 2.18772792 0.35469
tier 2 1920.032 2.36365416 0.35469
tier 2 1926.808 2.40337944 0.35469
tier 2 1932.904 2.16219024 0.35469
tier 2 1939.456 2.32392888 0.35469
tier 2 1945.392 2.10543984 0.35469
tier 2 1951.112 2.0288268 0.35469
tier 2 1957.208 2.16219024 0.35469
tier 2 1963.016 2.06003952 0.35469
tier 2 1968.688 2.01180168 0.35469
tier 2 1972.664 1.41024744 0.35469
tier 2 1974.6 0.68667984 0.35469
tier 2 1975.32 0.2553768 0.35469
tier 2 1975.32 0.0 0.35469
tier 2 1975.32 0.0 0.35469
tier 2 1975.384 0.02270016 0.35469
tier 2 1975.392 0.00283752 0.35469
tier 2 1975.464 0.02553768 0.35469
tier 2 1976.224 0.2695644 0.35469
tier 2 1978.704 1.0369624 0.41813
tier 2 1982.6 1.62903448 0.41813
tier 2 1987.864 2.20103632 0.41813
tier 2 1993.84 2.49874488 0.41813
tier 2 2000.184 2.65261672 0.41813
tier 2 2

In [26]:
sum(billing) # a few cents off

1118.6989048159994